In [31]:
import re
from collections import namedtuple

from tqdm.notebook import tqdm
import operator as op
from functools import partial
from typing import NamedTuple
from functools import reduce
mul = partial(reduce,op.mul)
assert mul(range(2,5)) == 24

def tenumerate(*args): return enumerate(tqdm(*args)) 


def flip(f):
    def inner(a,b): return f(b,a)
    return inner

In [32]:

class Amounts(NamedTuple):
    ore: int = 0
    clay: int = 0
    obsidian: int = 0
    geode: int = 0
    
    def _operate(self,other, operator):
        if isinstance(other, Amounts):
            return Amounts(*map(operator, self, other))
        else:
            return Amounts(*(operator(s,other) for s in self))
    
    def __add__(self, other): return self._operate(other,op.add)
    def __sub__(self, other): return self._operate(other,op.sub)
    def __mul__(self, other): return self._operate(other,op.mul)
    def __rsub__(self, other): return self._operate(other,flip(op.sub))


In [8]:
BP = namedtuple("Blueprint", "ore,clay,obsidian,geode")

In [73]:
def parse(fname):
    for line in open(fname):
        if line.strip():
            _,*ns= re.findall('[0-9]+', line)
            ns = map(int,ns)
            yield BP(ore=Amounts(ore=next(ns)), 
                               clay=Amounts(ore=next(ns)),
                               obsidian=Amounts(ore=next(ns),clay=next(ns)),
                                geode=Amounts(ore=next(ns),obsidian=next(ns)))
            
bps = list(parse("input"))

In [10]:
def evaluate(moves: list, bp: BP, t=24):
    minerals = Amounts()
    robots = Amounts(ore=1)
    breakpoint()
    while t > 0:
        #build robots
        move = moves.pop(0) if len(moves) > 0 else None
        if move:
            for i in range(4):
                minerals = minerals - bp[i]*move[i]
            
            if any(m < 0 for m in minerals):
                raise ValueError(minerals)
            
            new_robots = robots + move
            
        #collect
        minerals = minerals + robots
        
        #complete robots
        if move:
            robots = new_robots
        
        print(24-t+1,minerals)
        t-=1
    return minerals

moves=[
    None,None,Amounts(clay=1),
    None,Amounts(clay=1),None,
    Amounts(clay=1),None,None,
    None,Amounts(obsidian=1),Amounts(clay=1),
    None,None,Amounts(obsidian=1),
    None,None,Amounts(geode=1),
    None,None,Amounts(geode=1),
    None,None,None
]
#evaluate(moves,bps[0])

In [11]:
def minerals_spent_building_robots(robots, bp):
    minerals = Amounts()
    for i in range(4):
        minerals = minerals + bp[i]*robots[i]   
    return minerals

def can_build_robots(robots, minerals, bp):
    minerals = minerals - minerals_spent_building_robots(robots,bp)
    return all(m >= 0 for m in minerals)


In [63]:
bps[0]

Blueprint(ore=Amounts(ore=4, clay=0, obsidian=0, geode=0), clay=Amounts(ore=2, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=3, clay=14, obsidian=0, geode=0), geode=Amounts(ore=2, clay=0, obsidian=7, geode=0))

In [65]:
max(r[3] for r in bps[0])

0

In [66]:
def valid_moves(robots, minerals, bp):    
    # just moves of one robot at a time for now
    for i in range(4):
        robot = [0,0,0,0]
        robot[i] = 1
        robot = Amounts(*robot)
        if can_build_robots(robot, minerals, bp):
            yield robot

def useful_moves(robots, minerals, bp):
    for move in valid_moves(robots, minerals, bp):
        if is_useful_move(move, robots, minerals, bp):
            yield move

def is_useful_move(move, robots, minerals, bp):
    # once we have enough robot of a kind to reharvest this mineral lost in any move, don't make more robots of that type
    for i in range(3): # don't do this condition on geode, that we want to keep harvesting
        if move[i] > 0:
            max_of_this_mineral_we_can_spend_in_a_move = max(r[i] for r in bp)
            if robots[i] >= max_of_this_mineral_we_can_spend_in_a_move:
                return False
    return True
            
list(valid_moves(Amounts(), Amounts(2),bps[0]))

[Amounts(ore=0, clay=1, obsidian=0, geode=0)]

In [80]:
from math import ceil

def lower_bound(robots, minerals, bp, time_left):
    return minerals.geode + robots.geode*time_left

def upper_bound_1(robots, minerals, bp, time_left):
    # assuming we can build one geode robot per round until time_left given infinite resources
    # minerals.geode + robots.geode*time_left + (time_left-1) + (time_left-2) + (time_left -2) + ...
    # the series sum(i for i in range(m)) can be simplified to (m-1)*m/2)
    return minerals.geode + robots.geode*time_left + (time_left)*(time_left-1)//2

def upper_bound_2(robots, minerals, bp, time_left):
    ub = upper_bound_1(robots, minerals, bp, time_left)
    
    # unlike upper_bound_1, we don't assume infinite resources to build geode robots. We have to take
    # into account how much obsidian we can harevest (now assuming infinite resources to build obsidian bots)
    max_obsidian = minerals.obsidian + robots.obsidian * time_left + (time_left)*(time_left-1)//2
    
    # given that we have this max obsidian, the number of geode robots we can harvest may be smaller
    max_geode_robots = ceil(max_obsidian / bp.geode.obsidian) 
    
    # now, instead of computing the sum of the series time_left -> 0, we compute the sum 
    # from time_left -> (time_left - max_geode_robots)
    if max_geode_robots < time_left:
        ub = ub - (time_left - max_geode_robots)*(time_left-max_geode_robots+1) // 2
        
    return ub

def sum_range_to(n):
    return n*(n+1)//2

upper_bound = upper_bound_2

AssertionError: 

55

In [76]:
from collections import deque

def solve(bp: BP, t=24):

    best = 0
    # state is a tuple of (robots, minerals, time_left)
    todo = deque([(Amounts(ore=1), Amounts(), t)])
    visited = {}    
    state = None
    it = 0
    while todo:
        it += 1
        robots, minerals, time_left = state = todo.pop()
        
        lb = lower_bound(robots, minerals, bp, time_left)
            
        best = max(best, lb)
        
        vs = (robots, minerals)
        if visited.get(vs,0) >= time_left:
            continue
        else:
            visited[vs] = time_left
            
        if (it % 100_000 == 0): print(it, len(todo), len(visited), time_left, best)

        if time_left == 0:
            continue
         
        if upper_bound(robots, minerals, bp, time_left) <= best:
            continue
        
        # add the do nothing move
        todo.append((robots, minerals + robots, time_left - 1))
        
        for move in useful_moves(robots, minerals, bp):
            new_state = (
                robots + move,
                minerals - minerals_spent_building_robots(move, bp) + robots,
                time_left - 1
            )
            todo.append(new_state)

    print(f"Got best result in {it} iterations)
    return best

SyntaxError: EOL while scanning string literal (3895727705.py, line 44)

In [71]:
%time solve(bps[0])

100000 19 67658 6 9
CPU times: user 4.38 s, sys: 10.6 ms, total: 4.39 s
Wall time: 4.39 s


9

In [75]:
ans1 = sum((i+1)*solve(bp) for i,bp in tenumerate(bps))

  0%|          | 0/30 [00:00<?, ?it/s]

100000 13 56811 6 5
100000 15 65300 5 5
200000 10 101557 6 9
100000 14 58707 6 2
300000 24 130050 3 3
400000 20 165238 4 4
100000 14 61419 4 2
200000 13 106562 3 2
100000 16 70016 6 8
200000 7 118329 7 10
100000 5 62408 9 1
100000 18 59319 2 1
100000 17 66135 4 2
200000 14 116874 7 2
300000 15 164962 6 2
100000 8 60590 5 0
100000 9 64037 6 1
200000 10 111304 6 1
100000 14 66184 6 5
200000 17 113665 3 6
100000 14 66553 7 8
100000 12 63775 6 5
200000 18 120709 2 1
100000 5 64293 12 0
100000 29 63077 1 0
200000 17 117214 6 7
100000 20 59262 3 1


In [77]:
ans1

1150

In [78]:
ans2 = mul(solve(bp,t=32) for i,bp in tenumerate(bps[:3]))
ans2

  0%|          | 0/3 [00:00<?, ?it/s]

100000 16 73572 6 5
300000 13 187054 6 5
400000 17 233507 7 6
500000 19 273914 4 6
700000 12 347013 10 6
900000 30 429481 3 6
1000000 15 470630 7 6
1300000 31 591755 4 8
1800000 28 697881 4 8
1900000 15 728021 7 8
2000000 20 750865 5 8
2100000 10 769921 8 8
2300000 25 837427 7 8
2500000 25 903580 6 9
2600000 12 930242 9 9
2800000 19 996802 8 9
2900000 33 1024360 5 9
3000000 31 1050890 3 9
3300000 16 1134597 12 11
100000 24 86687 9 56
200000 25 165327 9 56
300000 24 234978 9 59
400000 23 299838 10 61
500000 19 365213 13 61
800000 17 538540 11 63
900000 21 580240 10 69
1000000 10 604333 14 69
1100000 15 638650 16 79
100000 15 88387 8 34
400000 14 300840 8 34
500000 22 372337 8 34
800000 9 554110 8 34
1200000 25 781094 7 36
1300000 28 823881 5 36
1400000 15 858033 10 36
1600000 21 944216 7 36
1700000 25 993490 8 36
1900000 28 1082037 7 43
2000000 20 1113607 8 43
2300000 34 1183354 7 43
2400000 14 1215121 10 43
2500000 17 1248190 8 43
2600000 17 1285028 11 43
2700000 24 1339181 9 43
280000

37367